In [1]:
import cv2 as cv
import numpy as np
import handTrackingModule as htm
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [5]:
#init
cap = cv.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)
detector = htm.handDetector(detectCon = 0.8, trackCon = 0.8)

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
# volume.GetMasterVolumeLevel()
print(volume.GetVolumeRange())
vol = 0
volRange = volume.GetVolumeRange()
#print(volRange)
minVol, maxVol = volRange[0], volRange[1]

#volume.SetMasterVolumeLevel(-20.0, None)

(-96.0, 0.0, 0.125)


In [4]:
while True:
    ret, frame = cap.read()
    frame = detector.findHand(frame)
    LMlist = detector.findPosition(frame)
    if len(LMlist):
        
        x1,y1 = LMlist[4][1],LMlist[4][2] #thumb
        x2,y2 = LMlist[8][1],LMlist[8][2] #pointing
        
        cv.circle(frame,(x1,y1),10,(255,0,255),-1)
        cv.circle(frame,(x2,y2),10,(255,0,255),-1)
        cv.line(frame,(x1,y1),(x2,y2),(0,255,255),3)
        
        length = math.hypot(x2-x1,y2-y1)
        #print(int(length)) #hand: max = 300, min = 20 | vol: max = 0, min = -96.00
        vol = np.interp(length,[20,300],[minVol,maxVol])
        #print(vol)
        volume.SetMasterVolumeLevel(vol, None)
        
    cv.imshow("Video",frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()
cv.waitKey(0)

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
